# Data Exploration & Simple Example for Field Boundary Delineation

In [1]:
import descarteslabs.workflows as wf
import numpy as np
import matplotlib.pyplot as plt
import math

/opt/conda/lib/python3.7/site-packages/descarteslabs/common/workflows/arrow_serialization/context.py:57: FutureWarning: 'pyarrow.SerializationContext' is deprecated and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  serialization_context = pa.SerializationContext()
/opt/conda/lib/python3.7/site-packages/descarteslabs/common/workflows/arrow_serialization/context.py:58: FutureWarning: 'pyarrow.register_default_serialization_handlers' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  pa.register_default_serialization_handlers(serialization_context)


In [2]:
def wgs_to_epsg(lat, lon):
    """
    Get the epsg code from a (lat, lon) location
    """
    utm_band = str((math.floor((lon + 180) / 6) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = "0" + utm_band
    if lat >= 0:
        epsg_code = "326" + utm_band
    else:
        epsg_code = "327" + utm_band
    return epsg_code

## Creating workflows image collections

We'll start by creating a workflows image collection for Sentinel-2. We will be focusing on one year of imagery (2019) for this exploration. More specifically we will be looking at imagery from around the growing season in our AOI.

In [3]:
year = 2020
start_datetime = f"{year}-05-01"
end_datetime = f"{year}-09-01"

In [4]:
ic = wf.ImageCollection.from_id(
    "sentinel-2:L1C",
    start_datetime=start_datetime,
    end_datetime=end_datetime,
    processing_level="surface"
)
s1 = wf.ImageCollection.from_id(
    "sentinel-1:GRD",
    start_datetime=start_datetime,
    end_datetime=end_datetime,
)



## Visualizing workflows image collections

Now let's look at the first image in the image collection using the workflows map and `.visualize()` method.

In [5]:
wf.map.center = [45.053594, -85.642160]

In [6]:
rgb = ic[0].pick_bands("red green blue")
rgb.visualize("Sentinel-2 RGB", scales=[[0, 0.4], [0, 0.4], [0, 0.4]])
s1_3_band = s1[0].pick_bands("vv vh vv")
s1_3_band.visualize("Sentinel-1 VV VH VV")

In [10]:
#wf.map

------------

## Examining the data more closely

Now that we have our Senintel-2 image collection let's pull the first few scenes locally to play with the data. We will be using the workflows map bounds to pull the data.

We'll center the map and fix the zoom level so we're all looking at the same imagery.

In [8]:
wf.map.center = [45.053594, -85.642160]
wf.map.zoom = 13

Now we can create a geocontext and access the imagery directly using the bounds of the map as a reference.

We grab the bounds of the workflows map using `wf.map.bounds`. We need to flatten and reorder these bounds to be compatible with the format required in a `wf.GeoContext`. The desired format is `(min_x, min_y, max_x, max_y)`

In [9]:
bounds_d = list(np.array(wf.map.bounds).flatten())
bounds = [bounds_d[1], bounds_d[0], bounds_d[3], bounds_d[2]]

IndexError: list index out of range

In [ ]:
utm_code = wgs_to_epsg(*wf.map.center)

In [ ]:
ctx = wf.GeoContext(
    bounds=bounds,
    bounds_crs="EPSG:4326",
    crs=f"EPSG:{utm_code}",
    resolution = 10.
)

Now we can actually get the data from platform to work with locally. We do this with `.compute()` and the geocontext we just created. We'll start by accessing the first 10 scenes from our image collection.

In [ ]:
result = ic[0:10].pick_bands("red green blue").compute(ctx)

In [ ]:
result

Our result has four primary pieces of information associated with it: `ndarray`, `properties`, `bandinfo`, and `geocontext`. `ndarray` is the actual image data stored in a `numpy.ndarray` object. `properties` contains a lot of useful information about the imagery you accessed. `bandinfo` is a dictionary of information about each band you requested. `geocontext` is the geocontext describing the data you accessed.

Let's look a the properties for one of the images we pulled in the Senintel-2 collection:

In [ ]:
result.properties[0].keys()

Let's also look at the shape of the underlying numpy ndarray.

In [ ]:
print("Sentinel-2 IC shape:\t" + str(result.ndarray.shape))

The dimensions of the array correspond to `(time, bands, pixel_x, pixel_y)`.

----------

Do the same for Sentinel-1 

In [ ]:
result_s1 = s1[0:10].pick_bands("vv vh vv").compute(ctx)

for prop in result_s1.properties:
    date, img_pass, orbit, img_id = (
        prop["date"],
        prop["pass"],
        prop["relative_orbit"],
        prop["id"],
    )
    print(f"{date}, {img_pass}, {orbit}, {img_id} \n")

------------

## Refining our Sentinel-2 workflow

We will need to find a way to deal with cloudy scenes in our Sentinel-2 image collection. The best way to go about this is to construct a series of composites (in this case monthly) leveraging as many cloud free pixels as possible.

First let's filter out any overly cloudly scenes using the `cloud_fraction` property. To do this we can use the `.filter()` method of our image collection. We must provide a lambda function to specify what fraction of the total pixels being covered by clouds we are okay with having in our imagery (in this case < 10%).

In [ ]:
ic_filtered = ic.filter(lambda img: img.properties["cloud_fraction"]<0.1)

cmask = ic_filtered.pick_bands("cloud-mask")
cmask = cmask > 0 

ic_masked = ic_filtered.mask(cmask)
monthly = ic_masked.groupby(lambda img: img.properties["date"].month).mean(axis="images")
result_rgb_monthly = monthly.pick_bands("red green blue").compute(ctx)

Finally let's take a look at the monthly cloud free composites.

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(9,12))
axs = axs.flatten()
for i in range(monthly_arr.shape[0]):
    axs[i].imshow(np.moveaxis(monthly_arr[i], 0, -1))
    axs[i].axis("off")
    axs[i].set_title("Month: " + str(result_rgb_monthly.properties[i]["group"]))

In [ ]:
s1_filtered = s1.filter(lambda img: img.properties["pass"] == "ASCENDING")
s1_monthly_max = s1_filtered.groupby(lambda img: img.properties["date"].month).max(
    axis="images"
)
s1_monthly_result = s1_monthly_max.pick_bands("vv vh vv").compute(ctx)

s1_monthly_arr = s1_monthly_result.ndarray

fig, axs = plt.subplots(4, 1, figsize=(9, 18))
axs = axs.flatten()
for i in range(s1_monthly_arr.shape[0]):
    axs[i].imshow(np.moveaxis(s1_monthly_arr[i], 0, -1))
    axs[i].axis("off")
    axs[i].set_title("Month: " + str(s1_monthly_result.properties[i]["group"]))


-------------

## Publishing our cloud free monthly workflow

In [ ]:
my_email = "penny.barnes@descarteslabs.com"
version = "1.0.0"

monthly.publish(
    f"{my_email}:sentinel_2_monthly_cloudfree_rgb",
    version,
    title=f"Sentinel-2 RGB Monthly Cloud Free Composite ({start_datetime} to {end_datetime})"
)

We can view all of our published workflows with this handy built-in widget:

In [ ]:
wf.interactive.WorkflowsBrowser()